In [4]:
import pandas as pd
import json
import requests

In [37]:
pd.set_option('display.max_rows', 1000)

In [5]:
def get_data(school_id):
    id = school_id
    url_ex = f'http://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid={id}'
    response = requests.get(url_ex)
    data = json.loads(response.text)
    num_of_professors = data['searchResultsTotal']
    num_divided = num_of_professors / 20
    
    if isinstance(num_divided, float) == True:
        num_divided = int(num_divided + 1)
        
    professors = []    
    for i in range(1, num_divided):
        id = school_id
        url = f'http://www.ratemyprofessors.com/filter/professor/?&page={i}&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid={id}'
        response = requests.get(url)
        data = json.loads(response.text)
        professors.append(data)
    new_list = []
    for i in range(len(professors)):
        prof_data = professors[i]['professors']
        new_list.append(prof_data)
    prof = []
    for i in range(len(new_list)):
        a = new_list[i]
        for j in range(len(a)):
            b = a[j]
            prof.append(b)
    
    prof_list = []
    
    for i in range(len(prof)):
            overall_rating = prof[i]['overall_rating']
            if overall_rating == 'N/A':
                continue
            else:
                school = prof[i]['institution_name']
                dept = prof[i]['tDept']
                name = prof[i]['tLname'] + ' ' + prof[i]['tFname']
                num_ratings = prof[i]['tNumRatings']
                rating = prof[i]['overall_rating']
                prof_tuple = (school, name, dept, float(rating), num_ratings)
                prof_list.append(prof_tuple)
    return prof_list

# Pulling and Parsing Public School's Data

In [30]:
#Biggest School in Each State
school_list =[4282,1256,1166,1530,352,1320,135,1003,1385,914,1309,1266,758,775,1596,724,1358,685,960,1058,1061,45,1391,1075,1087,1091,1094,1082,1101,1106,120,1112,440,1115,1117,1118,3071,1122,1270,1513,601,1257,617,1321,626,1249,1264,1231,825,1610]

50

In [ ]:
# loop pulling the data, putting it into a DF, 
# extracting school, rating(mean), rating Count(sum),
# creating a tuple and appending to a list

In [107]:
school_data=[]

def parse_data(data_list):
    for s in data_list:
        prof_list = get_data(s)
    
        df=pd.DataFrame(prof_list)
        df=df.rename(columns={0: "School", 1: "Professor", 2: "Department", 3:"Rating", 4:"Rating Count"})
        Rating = df['Rating'].mean()
        Rating_Count = df['Rating Count'].sum()
    
        school_tuple = (df['School'][0], Rating, Rating_Count)
        school_data.append(school_tuple)
    
    return school_data 

In [23]:
school_data_list = parse_data(school_list)

In [70]:
school_df=pd.DataFrame(school_data_list, columns = ['School', 'Rating', 'Rating Count'])

In [71]:
school_df

,School,Rating,Rating Count
0,University of Wyoming,3.726242,3091
1,University of Wisconsin - Madison,3.785536,47657
2,West Virginia University,3.805346,23026
3,University of Washington,3.784432,41845
4,George Mason University,3.777748,74811
5,University of Vermont,3.707608,17457
6,Brigham Young University,4.026193,85179
7,Texas A&M University at College Station,3.718053,31903
8,University of Tennessee - Knoxville,3.758922,38817
9,South Dakota State University,3.655382,4917


In [72]:
school_df.drop_duplicates(subset = "School", inplace=True)

In [80]:
school_df = school_df.reset_index()

In [84]:
school_df = school_df.drop(columns='index')

In [90]:
school_df['Public_or_Private']='public'

In [91]:
school_df

,School,Rating,Rating Count,Public_or_Private
0,University of Wyoming,3.726242,3091,public
1,University of Wisconsin - Madison,3.785536,47657,public
2,West Virginia University,3.805346,23026,public
3,University of Washington,3.784432,41845,public
4,George Mason University,3.777748,74811,public
5,University of Vermont,3.707608,17457,public
6,Brigham Young University,4.026193,85179,public
7,Texas A&M University at College Station,3.718053,31903,public
8,University of Tennessee - Knoxville,3.758922,38817,public
9,South Dakota State University,3.655382,4917,public


In [94]:
school_df.to_csv('Public_Schools_Data.csv')

# Pulling & Parsing Private School's Data

In [102]:
private_school_list = [399,953,1222,580,780,1275,137,148,1350,1339,298,774, 1072,278,355,1085,709,1576,1209,1258,799,464,400,1054,990,125,4002,33,234,1381,1147,1053,1277,1040,675,605,181,1075,87,1161,122,1139,1049,1156,1232,252,269,3965,402,83]
                       

In [103]:
len(private_school_list)

50

In [109]:
private_school_data_list = parse_data(private_school_list)

In [111]:
private_school_df=pd.DataFrame(private_school_data_list , columns = ['School', 'Rating', 'Rating Count'])

In [113]:
private_school_df['Public_or_Private']='private'

In [118]:
private_school_df

,School,Rating,Rating Count,Public_or_Private
0,Harvard University,3.961238,1671,private
1,Stanford University,3.941399,1963,private
2,Yale University,3.828696,1075,private
3,Massachusetts Institute of Technology,3.882051,1300,private
4,Princeton University,4.076571,804,private
5,University of Pennsylvania,3.777273,1757,private
6,Brown University,3.852632,2034,private
7,California Institute of Technology,3.680392,246,private
8,Duke University,3.848800,17231,private
9,Dartmouth College,3.870866,404,private


In [117]:
private_school_df.to_csv('Private_Schools_Data.csv')

In [ ]:
######################